In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

# Normal label
normal_label = "Benign"
# Base path where to find the dataset
base_path = "../data/IDS2018/original"
# Base path where processed dataset will be stored
export_path = "../data/IDS2018"
# Name of the file summarizing the preprocessing
info_fname = "ids2018_info.csv"
# File name of the cleaned/processed dataset
export_fname = "ids2018.csv"
# Used to track preprocessing steps
stats = defaultdict()
stats["n_dropped_cols"] = 0
stats["n_dropped_rows"] = 0
# Columns to drop before any analysis
cols_to_drop = [
    'Flow ID',
    'Src IP',
    'Dst IP',
    'Src Port',
    'Dst Port'
    'Protocol',
    'Timestamp',
]
num_cols = [
    'Tot Fwd Pkts',
    'Tot Bwd Pkts',
    'TotLen Fwd Pkts',
    'TotLen Bwd Pkts',
    'Fwd Pkt Len Max',
    'Fwd Pkt Len Min',
    'Fwd Pkt Len Mean',
    'Fwd Pkt Len Std',
    'Bwd Pkt Len Max',
    'Bwd Pkt Len Min',
    'Bwd Pkt Len Mean',
    'Bwd Pkt Len Std',
    'Flow Byts/s',
    'Flow Pkts/s',
    'Flow IAT Mean',
    'Flow IAT Std',
    'Flow IAT Max',
    'Flow IAT Min',
    'Fwd IAT Tot',
    'Fwd IAT Mean',
    'Fwd IAT Std',
    'Fwd IAT Max',
    'Fwd IAT Min',
    'Bwd IAT Tot',
    'Bwd IAT Mean',
    'Bwd IAT Std',
    'Bwd IAT Max',
    'Bwd IAT Min',
    'Fwd PSH Flags',
    'Fwd URG Flags',
    'Fwd Header Len',
    'Bwd Header Len',
    'Fwd Pkts/s',
    'Bwd Pkts/s',
    'Pkt Len Min',
    'Pkt Len Max',
    'Pkt Len Mean',
    'Pkt Len Std',
    'Pkt Len Var',
    'FIN Flag Cnt',
    'SYN Flag Cnt',
    'RST Flag Cnt',
    'PSH Flag Cnt',
    'ACK Flag Cnt',
    'URG Flag Cnt',
    'CWE Flag Count',
    'ECE Flag Cnt',
    'Down/Up Ratio',
    'Pkt Size Avg',
    'Fwd Seg Size Avg',
    'Bwd Seg Size Avg',
    'Subflow Fwd Pkts',
    'Subflow Fwd Byts',
    'Subflow Bwd Pkts',
    'Subflow Bwd Byts',
    'Fwd Act Data Pkts',
    'Fwd Seg Size Min',
    'Active Mean',
    'Active Std',
    'Active Max',
    'Active Min',
    'Idle Mean',
    'Idle Std',
    'Idle Max',
    'Idle Min'
]

In [4]:
df = pd.DataFrame()
for f in os.listdir(base_path):
    chunk = pd.read_csv(os.path.join(base_path, f))
    chunk.columns = chunk.columns.str.strip()
    chunk.loc[:, chunk.columns != "Label"] = chunk.loc[:, chunk.columns != "Label"].apply(pd.to_numeric, errors="coerce")   
    chunk.drop(cols_to_drop, axis=1, errors="ignore")
    df = pd.concat((df, chunk))
    print(f)
print(stats)
df.to_csv(export_path + "/ids2018_merged.csv", index=False)
df.head(5)

Friday-02-03-2018_TrafficForML_CICFlowMeter.csv


C:\Users\verdi\anaconda3\envs\anomaly_detection\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv
defaultdict(None, {'n_dropped_cols': 0, 'n_dropped_rows': 0, 'dropped_cols': '', 'n_instances': 16233002, 'n_features': 83, 'anomaly_ratio': '0.1693'})


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Flow ID,Src IP,Src Port,Dst IP
0,443,6,NaN,141385,9,7,553,3773.0,202,0,...,0.0,0.0,0.0,0.0,0.0,Benign,NaN,NaN,NaN,NaN
1,49684,6,NaN,281,2,1,38,0.0,38,0,...,0.0,0.0,0.0,0.0,0.0,Benign,NaN,NaN,NaN,NaN
2,443,6,NaN,279824,11,15,1086,10527.0,385,0,...,0.0,0.0,0.0,0.0,0.0,Benign,NaN,NaN,NaN,NaN
3,443,6,NaN,132,2,0,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,Benign,NaN,NaN,NaN,NaN
4,443,6,NaN,274016,9,13,1285,6141.0,517,0,...,0.0,0.0,0.0,0.0,0.0,Benign,NaN,NaN,NaN,NaN


In [2]:
df = df.drop(["Flow ID", "Src IP", "Src Port", "Dst IP", "Dst Port", "Protocol", "Timestamp"], axis=1)
df.to_csv(export_path + "/ids2018_merged.csv", index=False)
df.columns

NameError: name 'df' is not defined

In [2]:
df = pd.read_csv(export_path + "/ids2018_merged.csv")
stats["dropped_cols"] = ""
stats["n_dropped_cols"] = 0
stats["n_dropped_rows"] = 0
stats["n_instances"] = len(df)
stats["n_features"] = df.shape[1] - 1
stats["anomaly_ratio"] = "{:2.4f}".format((df["Label"] != normal_label).sum() / len(df))
df.head(5)

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,141385.0,9.0,7.0,553.0,3773.0,202.0,0.0,61.444444,87.534438,1460.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,281.0,2.0,1.0,38.0,0.0,38.0,0.0,19.000000,26.870058,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,279824.0,11.0,15.0,1086.0,10527.0,385.0,0.0,98.727273,129.392497,1460.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,132.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,274016.0,9.0,13.0,1285.0,6141.0,517.0,0.0,142.777778,183.887722,1460.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [3]:
df.dtypes

Flow Duration      float64
Tot Fwd Pkts       float64
Tot Bwd Pkts       float64
TotLen Fwd Pkts    float64
TotLen Bwd Pkts    float64
                    ...   
Idle Mean          float64
Idle Std           float64
Idle Max           float64
Idle Min           float64
Label               object
Length: 77, dtype: object

## Inner class imbalance
Between anomalies, there is a strong class imbalance

In [4]:
# Original class imbalance within attacks themselves
mask = df["Label"] != normal_label
original_ad_ratios = pd.DataFrame(
    pd.concat(
        (df[mask]["Label"].value_counts(),
        df[mask]["Label"].value_counts() / len(df[mask])), axis=1),
)
original_ad_ratios.to_csv(export_path + "/ids2018_anomaly_labels_ratio.csv")
original_ad_ratios.columns = ["Count", "Ratio"]
original_ad_ratios

,Count,Ratio
DDOS attack-HOIC,686012,0.249614
DDoS attacks-LOIC-HTTP,576191,0.209654
DoS attacks-Hulk,461912,0.168072
Bot,286191,0.104134
FTP-BruteForce,193360,0.070356
SSH-Bruteforce,187589,0.068257
Infilteration,161934,0.058922
DoS attacks-SlowHTTPTest,139890,0.050901
DoS attacks-GoldenEye,41508,0.015103
DoS attacks-Slowloris,10990,0.003999


In [5]:
# Group DoS attacks
mask = df["Label"].str.startswith("DoS")
df.loc[mask, "Label"] = "DoS"

# Group DDoS attacks
mask = df["Label"].str.startswith("DDoS")
df.loc[mask, "Label"] = "DDoS"
mask = df["Label"].str.startswith("DDOS")
df.loc[mask, "Label"] = "DDoS"

# Group Web attacks
mask = df["Label"].str.startswith("Brute Force")
df.loc[mask, "Label"] = "Web Attack"
mask = df["Label"].str.startswith("SQL")
df.loc[mask, "Label"] = "Web Attack"

Found 49 rows that are duplicates of the header row

In [29]:
df[df["Label"] == "Label"]

0

In [8]:
# Updated class imbalance
mask = df["Label"] != normal_label
mod_ad_ratios = pd.DataFrame(
    pd.concat(
        (df[mask]["Label"].value_counts(),
        df[mask]["Label"].value_counts() / len(df[mask])), axis=1),
)
mod_ad_ratios.columns = ["Count", "Ratio"]
mod_ad_ratios

,Count,Ratio
DDoS,1263933,0.459897
DoS,654300,0.238075
Bot,286191,0.104134
FTP-BruteForce,193360,0.070356
SSH-Bruteforce,187589,0.068257
Infilteration,161934,0.058922
Web Attack,928,0.000338
Label,59,0.000021


In [9]:
# Group DoS attacks
mask = df["Label"].str.startswith("DoS")
df.loc[mask, "Label"] = "DoS"

# Group DDoS attacks
mask = df["Label"].str.lower().str.startswith("ddos")
df.loc[mask, "Label"] = "DDoS"

# Group Web attacks
mask = df["Label"].str.startswith("Web Attack")
df.loc[mask, "Label"] = "Web Attack"

In [7]:
# Updated class imbalance
mask = df["Label"] != normal_label
mod_ad_ratios = pd.DataFrame(
    pd.concat(
        (df[mask]["Label"].value_counts(),
        df[mask]["Label"].value_counts() / len(df[mask])), axis=1),
)
mod_ad_ratios.columns = ["Count", "Ratio"]
mod_ad_ratios

,Count,Ratio
DDoS,1263933,0.459897
DoS,654300,0.238075
Bot,286191,0.104134
FTP-BruteForce,193360,0.070356
SSH-Bruteforce,187589,0.068257
Infilteration,161934,0.058922
Web Attack,928,0.000338
Label,59,0.000021


## Check unique values
Drop columns with unique values

In [8]:
uniq_cols = df.columns[df.nunique() <= 1].tolist()
stats["n_unique_cols"] = len(uniq_cols)
if uniq_cols:
    print("Found {} columns with unique values: {}".format(len(uniq_cols), uniq_cols))
    stats["unique_cols"] = ", ".join([str(col) for col in uniq_cols])
    df.drop(uniq_cols, axis=1, inplace=True)
    stats["n_dropped_cols"] += len(uniq_cols)
    uniq_cols = df.columns[df.nunique() <= 1].tolist()
assert len(uniq_cols) == 0, "Found {} columns with unique values: {}".format(len(uniq_cols), uniq_cols)
print("Columns are valid with more than one distinct value")

Found 8 columns with unique values: ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg']
Columns are valid with more than one distinct value


## Check for NaN/invalid values
First, find the columns with NaN values. Further processing will be required if we find any.

In [9]:
#n_dropped = len(df[df["Flow Duration"].isna()])
#df = df[df["Flow Duration"].isna()].dropna()
n_dropped = len(df[df["Flow Duration"].isna()])
stats["n_dropped_rows"] += n_dropped
df = df.drop(index=df[df["Flow Duration"].isna()].index)
print("Dropped {} rows".format(n_dropped))

Dropped 59 rows


In [10]:
# Replacing INF values with NaN
df = df.replace([-np.inf, np.inf], np.nan)
nan_cols = df.columns[df.isna().sum() > 0].tolist()
stats["n_nan_cols"] = len(nan_cols)
if nan_cols:
    stats["nan_cols"] = ", ".join([str(col) for col in nan_cols])
print("Found NaN columns: {}".format(nan_cols))

Found NaN columns: ['Flow Byts/s', 'Flow Pkts/s']


Having found two columns with NaN values, we must investigate further before taking any decision.

Dropping them seems risky because we would also lose anomalies which are already scarce and important for evaluation.

In [11]:
df[df.isna().any(axis=1)]["Label"].value_counts()

Benign            94459
Infilteration      1295
FTP-BruteForce        6
Name: Label, dtype: int64

Dropping them seems risky since we also drop anomalies which are already scarce

In [12]:
# Check different values in Flow Duration
print(df[df.isna().any(axis=1)]["Flow Duration"].unique())
# Count number of nan instances when `Flow Duration` > 0
df[df["Flow Duration"] > 0].isna().sum().sum()

[0.]


0

`Flow Bytes/s` and `Flow Packets/s` must be computed from `Flow Duration`. When the latter columns is zero, a division by zero occurs and the first two columns have NaN values. Zero values in `Flow Duration` are probably due to a lack of precision in the data type used. They must be associated with flows that lasted nanoseconds. Hence, we can convert the NaN rows to zeros.

In [13]:
n_dropped = df[nan_cols].isna().sum()[0]
df = df.fillna(0)
print("Replaced {} rows or {:2.4f}% of original data".format(n_dropped, n_dropped / len(df)))
remaining_nans = df.isna().sum().sum()
assert remaining_nans == 0, "There are still {} NaN values".format(remaining_nans)

Replaced 95760 rows or 0.0059% of original data


## Check for negative values
Most of the features should be strictly positive. For instance, a packet with a negative number of bytes makes no sense.

In [14]:
num_cols = df.select_dtypes(exclude="object").columns
mask = (df[num_cols] < 0).sum() > 0
neg_cols = df[num_cols].columns[mask]
stats["n_negative_cols"] = len(neg_cols)
stats["negative_cols"] = ", ".join(neg_cols)
print("Found {} columns with negative values: {}".format(len(neg_cols), neg_cols))

Found 11 columns with negative values: Index(['Flow Duration', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Max',
       'Fwd IAT Min', 'Init Fwd Win Byts', 'Init Bwd Win Byts'],
      dtype='object')


In [15]:
neg_df = pd.DataFrame(
    pd.concat((
        (df[neg_cols] < 0).sum(),
        (df[neg_cols] < 0).sum() / len(df)
    ), axis=1)
)
neg_df.columns = ["Count", "Ratio"]
neg_df = neg_df.sort_values("Count", ascending=False)
neg_df

,Count,Ratio
Init Bwd Win Byts,8255549,5.085676e-01
Init Fwd Win Byts,4432608,2.730625e-01
Flow IAT Min,15,9.240469e-07
Fwd IAT Min,15,9.240469e-07
Flow Duration,14,8.624437e-07
Flow Pkts/s,14,8.624437e-07
Flow IAT Mean,14,8.624437e-07
Fwd IAT Tot,14,8.624437e-07
Fwd IAT Mean,14,8.624437e-07
Flow IAT Max,3,1.848094e-07


In [16]:
print((df["Init Bwd Win Byts"][df["Init Bwd Win Byts"] < 0]).unique())
print((df["Init Fwd Win Byts"][df["Init Fwd Win Byts"] < 0]).unique())
#df[df["Init_Win_bytes_backward"] < 0]

[-1.]
[-1.]


In [17]:
# Drop `Init_Win_bytes_forward` and `Init_Win_bytes_backward` because too many of their values are equal to -1 which makes no sense.
to_drop = neg_df[neg_df["Ratio"] > 0.01].index.tolist()
df = df.drop(to_drop, axis=1)
neg_df = neg_df.drop(to_drop)
stats["n_dropped_cols"] += len(to_drop)
stats["dropped_cols"] = stats["dropped_cols"] + ", ".join(to_drop)
num_cols = df.select_dtypes(include=np.number).columns
print("Dropped {} columns: {}".format(len(to_drop), to_drop))

Dropped 2 columns: ['Init Bwd Win Byts', 'Init Fwd Win Byts']


In [18]:
df[(df[num_cols] < 0).any(1)]["Label"].value_counts()

Benign    15
Name: Label, dtype: int64

The remaining invalid values are associated only to 15 benign rows. Removing them is probably the safest solution here.

In [19]:
df[(df[num_cols] < 0).any(1)]

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
3030242,5.132268e+06,10.0,8.0,577.0,1440.0,109.0,0.0,57.7,49.888877,248.0,...,20.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Benign
12720606,-1.885050e+11,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.974748e+11,1.412517e+11,3.248070e+11,2.175600e+10,Benign
12720607,-7.487700e+10,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,3.056785e+11,9.931048e+10,4.508620e+11,2.399340e+11,Benign
12720608,-4.834000e+09,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,3.759760e+11,2.622479e+11,8.464580e+11,2.129100e+10,Benign
12720609,-8.282200e+11,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Benign
12720610,-5.787680e+11,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,3.955714e+11,1.897622e+11,7.228210e+11,6.352000e+09,Benign
12720611,-6.990560e+11,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,2.593934e+11,1.760081e+11,6.060520e+11,5.196000e+10,Benign
12720612,-5.533120e+11,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,2.520679e+11,1.613787e+11,5.622340e+11,4.375000e+09,Benign
12720888,-1.101160e+11,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,3.553386e+11,2.547013e+11,8.637480e+11,1.506900e+10,Benign
12722487,-6.420520e+11,163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,3.560329e+11,2.347642e+11,9.684340e+11,2.054000e+09,Benign


When Flow Duration < 0, multiple columns are negative. Since these rows are only associated with BENIGN flows, we can drop them.

In [20]:
n_dropped = (df["Flow Duration"] < 0).sum()
stats["n_dropped_rows"] += n_dropped
df = df[df["Flow Duration"] >= 0]
print("Dropped {} rows".format(n_dropped))
# assert len(df[(df[num_cols] < 0).any(1)]) == 0, "there are still negative rows"

Dropped 14 rows


In [21]:
df = df.drop(index=df[(df[num_cols] < 0).any(1)].index)
stats["n_dropped_rows"] += 1
assert len(df[(df[num_cols] < 0).any(1)]) == 0, "there are still negative rows"

In [22]:
df["Category"] = df["Label"]
df["Label"] = df["Label"].apply(lambda x: 0 if x == normal_label else 1)
df["Label"] = df["Label"].astype(np.uint8)

## Normalize attributes

In [23]:
num_cols = df.select_dtypes(include=[np.number]).columns
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
assert np.allclose(df[num_cols].max(axis=0).to_numpy(), 1.), "Found values different than 1."
assert np.allclose(df[num_cols].min(axis=0).to_numpy(), 0.), "Found values lesser than 0."
print("Data is scaled between 0 and 1")

Data is scaled between 0 and 1


In [24]:
stats["n_final_features"] = df.shape[1] - 2
stats["n_final_rows"] = df.shape[0]
stats["final_anomaly_ratio"] = (df["Label"] != 0).sum() / len(df)
stats

defaultdict(None,
            {'n_dropped_cols': 10,
             'n_dropped_rows': 74,
             'dropped_cols': 'Init Bwd Win Byts, Init Fwd Win Byts',
             'n_instances': 16233002,
             'n_features': 76,
             'anomaly_ratio': '0.1693',
             'n_unique_cols': 8,
             'unique_cols': 'Bwd PSH Flags, Bwd URG Flags, Fwd Byts/b Avg, Fwd Pkts/b Avg, Fwd Blk Rate Avg, Bwd Byts/b Avg, Bwd Pkts/b Avg, Bwd Blk Rate Avg',
             'n_nan_cols': 2,
             'nan_cols': 'Flow Byts/s, Flow Pkts/s',
             'n_negative_cols': 11,
             'negative_cols': 'Flow Duration, Flow Pkts/s, Flow IAT Mean, Flow IAT Max, Flow IAT Min, Fwd IAT Tot, Fwd IAT Mean, Fwd IAT Max, Fwd IAT Min, Init Fwd Win Byts, Init Bwd Win Byts',
             'n_final_features': 66,
             'n_final_rows': 16232928,
             'final_anomaly_ratio': 0.16930001784028118})

## Store processed dataset to CSV

In [25]:
df.to_csv(export_path + "/" + export_fname, index=False)
print("Processed data saved under: {}".format(export_path + "/" + export_fname))

Processed data saved under: ../data/IDS2018/original/ids2018.csv


## Store basic information

In [26]:
stats_df = pd.DataFrame(stats, index=[0])
stats_df.to_csv(export_path + "/" + info_fname, index=False)
print("Processing summary saved under: {}".format(base_path + "/" + info_fname))
stats_df

Processing summary saved under: ../data/IDS2018/original/ids2018_info.csv


,n_dropped_cols,n_dropped_rows,dropped_cols,n_instances,n_features,anomaly_ratio,n_unique_cols,unique_cols,n_nan_cols,nan_cols,n_negative_cols,negative_cols,n_final_features,n_final_rows,final_anomaly_ratio
0,10,74,"Init Bwd Win Byts, Init Fwd Win Byts",16233002,76,0.1693,8,"Bwd PSH Flags, Bwd URG Flags, Fwd Byts/b Avg, ...",2,"Flow Byts/s, Flow Pkts/s",11,"Flow Duration, Flow Pkts/s, Flow IAT Mean, Flo...",66,16232928,0.1693
